In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
)

# components for one episode
from scai.custom_chat_models.crfm import crfmChatLLM # model
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

# meta prompt 
from scai.modules.meta_prompt.base import MetaPromptModel # model
from scai.modules.meta_prompt.prompts import META_PROMPTS # prompts

# user 
from scai.modules.user.base import UserModel # model
from scai.modules.user.prompts import USER_PROMPTS # prompts

# assistant
from scai.modules.assistant.base import AssistantModel # model
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS # prompts
from scai.modules.assistant.models import AssistantPrompt # prompt

# task
from scai.modules.task.prompts import TASK_PROMPTS

In [15]:
# sample task
task_prompt = TASK_PROMPTS["task_prompt_1"]
# sample users 
user_prompt = USER_PROMPTS["user_prompt_1"]
# sample assistant
assistant_prompt = ASSISTANT_PROMPTS["assistant_prompt_1"]
# sample meta prompt
meta_prompt = META_PROMPTS["meta_prompt_1"]

In [40]:
# initialise buffer 
buffer = CustomConversationBufferWindowMemory(system_k=2, 
                                              chat_k=2, 
                                              user_k=2, 
                                              assistant_k=1,
                                              ) # how many turns are stored will be used

system_message = "You are a helpful AI assistant."

# initialise task
print("TASK")
task = task_prompt.content
print(task)
print("=====================================")
# initialise user 
print("USER")
user_system_prompt = SystemMessagePromptTemplate.from_template(user_prompt.content)
user_chat_prompt = ChatPromptTemplate.from_messages([user_system_prompt])
print(user_chat_prompt.format(persona=user_prompt.persona, task=task))
print("=====================================")
# initialise assistant
print("ASSISTANT")
assistant_system_prompt = SystemMessagePromptTemplate.from_template(assistant_prompt.content)
assistant_chat_prompt = ChatPromptTemplate.from_messages([assistant_system_prompt])
print(assistant_chat_prompt.format(system_message=system_message))
print("=====================================")
print("BUFFER")
buffer.save_context(user={"role": "user", "content": "hi"}, 
                    assistant={"role": "assistant", "content": "hi"})
print(buffer.full_memory.messages)

TASK
Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
USER
System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
ASSISTANT
System: You are a helpful AI assistant.
BUFFER


ValueError: One input key expected got ['content', 'role']

In [38]:
"""The User."""
from typing import (
    Any,
    Dict,
    List, 
    Optional
)

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
)

from scai.modules.user.models import UserPrompt
from scai.modules.user.prompts import USER_PROMPTS
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

from scai.modules.task.models import TaskPrompt

from langchain.chains.llm import LLMChain

class UserModel():
    """Chain for applying the User.

    Example:
        .. code-block:: python

            

    """

    def __init__(self, llm) -> None:
        self.llm = llm

    @classmethod
    def get_prompts(
        cls, names: Optional[List[str]] = None
    ) -> List[UserPrompt]:
        return list(USER_PROMPTS.values()) if names is None else [USER_PROMPTS[name] for name in names]
    
    @classmethod 
    def get_template(
        cls, name: str
    ) -> str:
        """Get prompt (i.e. meta system message) based on name."""
        return cls.get_prompts([name])[0].content
    
    def run(
        self,
        buffer: CustomConversationBufferWindowMemory,
        user_prompt: UserPrompt,
        task_prompt: TaskPrompt,
        max_tokens: int = 100, # max tokens to generate
    ) -> str:
        """Run user."""
        # user system message
        user_system_prompt = SystemMessagePromptTemplate.from_template(user_prompt.content) 
        # get chat history
        chat_history_prompts = [m for m in buffer.load_memory_variables(var_type="chat")['history']]      
        # prompt to generate next completion based on history
        generate_next = """Provide a response using no more than {max_tokens} tokens."""
        generate_next_prompt = HumanMessagePromptTemplate.from_template(generate_next)
        # build prompt template
        user_chat_prompt = ChatPromptTemplate.from_messages([user_system_prompt, *chat_history_prompts, generate_next_prompt])
        print(user_chat_prompt.format(persona=user_prompt.persona,
                                      task=task_prompt.content,
                                      max_tokens=user_prompt.max_tokens))

        
   
        # run user
        # chain = LLMChain(llm=self.llm, prompt=user_chat_prompt)
        # response = chain.run(persona=user_prompt.persona,
        #                      task=task_prompt.content,
        #                      chat_history=chat_history,
        #                      max_tokens=max_tokens,
        #                      stop=["System:"])
        # return response
    


In [39]:
user_model = UserModel(llm='chat_llm')
user_response = user_model.run(user_prompt=user_prompt, 
                               task_prompt=task_prompt, 
                               buffer=buffer)

print(user_response)

System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a few other Users and Assitant(s) to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: Provide a response using no more than 100 tokens.
None


In [20]:
assistant_model = AssistantModel(llm='chat_llm')
assistant_prompt.content = system_message # this will be updated as we walk through the conversation
assistant_response = assistant_model.run(assistant_prompt=assistant_prompt, buffer=buffer)
print(assistant_response)

System: You are a helpful AI assistant.


In [21]:
meta_model = MetaPromptModel(llm='chat_llm')
meta_response = meta_model.run(meta_prompt=meta_prompt, buffer=buffer)
print(meta_response)

System: Assistant has just had the below interaction with the User. Both Assistant and User followed their system message closely to answer a question together. 
        
Your job is to critique Assistant's performance and provide new system message so that Assistant can correctly and quickly respond in the future.

#### Chat History Starts #### 



#### Chat History Ends #### 

Your first task is to critique Assistan's performance: 
1. What could Assistant have done better to satisfy User? 
2. Was Assistant effective in answering the question posed? 
3. Was Assistant receptive to User's feedback? 
4. Did Assistant satisfy User as correctly and quickly as possible? 

You should indicate this feedback with “Critique: ...”.

Your next task is to revise Assistant's system message. Older system messages including previous revisions are shown below. 

#### Assistant's old system message(s) Start #### 



#### Assistant's old system message(s) End #### 

You must ensure that the Assistant ca

In [ ]:
# # prompting a custom chat model
# from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

# chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
#                    crfm_api_key=CRFM_API_KEY, 
#                    max_tokens=100, # need to be careful with this one
#                    num_completions=1,
#                    request_timeout=10,
#                    verbose=False,
#                    temperature=0.9,
#                    )